# 🔄 Spotify Library Sync

This notebook downloads your Spotify library and saves it locally for offline analysis.

**What it does:**
- ✅ Fetches all your playlists (owned only)
- ✅ Fetches your Liked Songs (❤️ master playlist)
- ✅ Downloads track and artist metadata
- ✅ Saves everything to `../data/` as parquet files
- ✅ Incremental updates (only fetches changes)

**Run this first!** Then use `02_analyze_library.ipynb` for analysis.

## 1️⃣ Setup

Install dependencies and configure credentials.

In [1]:
# Install dependencies (run once)
%pip install -q pandas spotipy pyarrow tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Add project to path
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Project root: {PROJECT_ROOT}")

✅ Project root: /Users/aryamaan/Desktop/Projects/spotim8_repo


In [3]:
import os

# ╔════════════════════════════════════════════════════════════════╗
# ║  CONFIGURE YOUR SPOTIFY CREDENTIALS HERE                       ║
# ║                                                                 ║
# ║  Get these from: https://developer.spotify.com/dashboard       ║
# ║  1. Create an app                                              ║
# ║  2. Add redirect URI: http://127.0.0.1:8888/callback           ║
# ║  3. Copy Client ID and Client Secret below                     ║
# ╚════════════════════════════════════════════════════════════════╝

os.environ["SPOTIPY_CLIENT_ID"] = "YOUR_CLIENT_ID"          # <-- Replace this          
os.environ["SPOTIPY_CLIENT_SECRET"] = "YOUR_CLIENT_SECRET"  # <-- Replace this 
os.environ["SPOTIPY_REDIRECT_URI"] = "http://127.0.0.1:8888/callback"

print("✅ Credentials configured!")
print("   (Make sure to replace YOUR_CLIENT_ID and YOUR_CLIENT_SECRET)")

✅ Credentials configured!
   (Make sure to replace YOUR_CLIENT_ID and YOUR_CLIENT_SECRET)


## 2️⃣ Connect to Spotify

This will open a browser window for authentication on first run.

In [4]:
from spotim8 import Spotim8
from spotim8.catalog import CacheConfig

# Data directory (stores downloaded data)
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Initialize client with caching
sf = Spotim8.from_env(
    progress=True,
    cache=CacheConfig(dir=DATA_DIR)
)

print(f"✅ Connected to Spotify!")
print(f"📁 Data will be saved to: {DATA_DIR}")

✅ Connected to Spotify!
📁 Data will be saved to: /Users/aryamaan/Desktop/Projects/spotim8_repo/data


/Users/aryamaan/Desktop/Projects/spotim8_repo/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 3️⃣ Sync Your Library

This fetches your playlists and tracks. First run may take a few minutes.

In [5]:
# Sync library (incremental - only fetches changes)
stats = sf.sync(
    owned_only=True,           # Only your playlists, not followed ones
    include_liked_songs=True   # Include Liked Songs as master playlist
)

print(f"\n📊 Sync complete!")

🔄 Starting library sync...
✅ All playlists up to date!
✅ Sync complete! Checked 126 playlists, updated 0, added 0 track entries

📊 Sync complete!


## 4️⃣ Build Full Data Tables

Now let's build all the detailed tables (tracks, artists, etc.)

In [6]:
# Fetch all data tables (uses cache if available)
print("📥 Building data tables...\n")

playlists = sf.playlists()
print(f"✅ Playlists: {len(playlists):,}")

playlist_tracks = sf.playlist_tracks()
print(f"✅ Playlist-track links: {len(playlist_tracks):,}")

tracks = sf.tracks()
print(f"✅ Unique tracks: {len(tracks):,}")

track_artists = sf.track_artists()
print(f"✅ Track-artist links: {len(track_artists):,}")

artists = sf.artists()
print(f"✅ Artists: {len(artists):,}")

# Build the wide table (everything joined)
library = sf.library_wide()
print(f"✅ Library wide table: {len(library):,} rows")

📥 Building data tables...

✅ Playlists: 555
✅ Playlist-track links: 31,164
✅ Unique tracks: 5,232
✅ Track-artist links: 8,414
✅ Artists: 2,597
✅ Library wide table: 31,207 rows


## 5️⃣ View Your Data

In [7]:
# Show status summary
sf.print_status()


        SPOTIFYFRAMES DATA STATUS
📁 Cache directory: /Users/aryamaan/Desktop/Projects/spotim8_repo/data
👤 User: 31iol2qamank24owygxo7kpq533y
🕐 Last sync: 2025-12-19T01:17:45.099294+00:00

📊 Cached data:
   • Playlists: 555
   • Playlist tracks: 31,164
   • Unique tracks: 5,232
   • Track-artist links: 8,414
   • Artists: 2,597



In [8]:
# Preview playlists
print("📂 Your Playlists:")
playlists[["name", "track_count", "is_liked_songs", "is_owned"]].head(15)

📂 Your Playlists:


,name,track_count,is_liked_songs,is_owned
0,❤️ Liked Songs,5077,True,True
1,Trapsoul,8,False,True
2,Dec’25,112,False,True
3,Trance,7,False,True
4,ADHD Study Music 🧠,300,False,False
5,DarkThots🌑🔪,28,False,True
6,Funky / Hip Hop / Chilled / Beats,361,False,False
7,Funky hip hop soul grooves. Music for the soul,1498,False,False
8,2013-2015 EDM Hits,54,False,False
9,Pop ig dont deep it,174,False,True


In [9]:
# Preview tracks
print("🎵 Sample Tracks:")
tracks[["name", "album_name", "popularity", "duration_ms"]].head(10)

🎵 Sample Tracks:


,name,album_name,popularity,duration_ms
0,Chamber Of Reflection,Salad Days,83,231723
1,Can't Feel My Face,Beauty Behind The Madness,84,213520
2,Walking On A Dream,Walking On A Dream (10th Anniversary Edition),86,198440
3,Teenage Blue,Illuminaughty,64,234968
4,we fell in love in october,we fell in love in october / October Passed Me By,87,184153
5,fwb,punk2,48,137751
6,Cult Classic,Cult Classic,53,107520
7,"VIRGINIA Boy (feat. Tyler, The Creator) - Remix",Piece By Piece - Music from the Motion Picture,50,128843
8,When I Was Your Man,Unorthodox Jukebox,89,213826
9,Don't,T R A P S O U L,84,198294


In [10]:
# Preview artists
print("🎤 Top Artists (by followers):")
artists.nlargest(10, "followers")[["name", "genres", "popularity", "followers"]]

🎤 Top Artists (by followers):


,name,genres,popularity,followers
926,Arijit Singh,"[hindi pop, bollywood, desi, bangla pop]",92,168225553
1492,Taylor Swift,[],100,148145192
1024,Ed Sheeran,[soft pop],90,123750238
1171,Billie Eilish,[],93,120745503
1,The Weeknd,[],96,115575232
427,Ariana Grande,[pop],95,108437388
1165,Eminem,"[rap, hip hop]",91,105865520
126,Bad Bunny,"[reggaeton, trap latino, urbano latino, latin]",98,104932669
75,Drake,[rap],98,104906249
463,Justin Bieber,[],93,85869099


## 6️⃣ Check Saved Files

In [11]:
# List saved files
print(f"📁 Files in {DATA_DIR}:\n")
for f in sorted(DATA_DIR.glob("*.parquet")):
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name:30} {size_kb:>8.1f} KB")

📁 Files in /Users/aryamaan/Desktop/Projects/spotim8_repo/data:

   artists.parquet                   196.2 KB
   library_wide.parquet             1644.4 KB
   playlist_tracks.parquet           562.8 KB
   playlists.parquet                 104.0 KB
   track_artists.parquet             215.2 KB
   tracks.parquet                    606.8 KB


---

## ✅ Done!

Your library is now saved locally. Next steps:

1. **Analyze**: Open `02_analyze_library.ipynb` for visualizations
2. **Playlist Analysis**: Open `03_playlist_analysis.ipynb` for genre clustering
3. **Re-sync**: Run this notebook again anytime to fetch new changes

The data is cached, so future runs are fast!